# Model 3 — Churn Propensity: Training & Evaluation

This notebook trains, evaluates, and explains a **churn propensity model** for a telecommunications
customer base (Telco Customer Churn dataset).  

The pipeline follows these steps:

1. Load engineered customer features (`customer_features.parquet`)
2. Stratified 80/20 train/test split
3. Cross-validate three models: Naive baseline, Logistic Regression, LightGBM
4. Calibrate LightGBM probabilities with Isotonic Regression
5. Evaluate: ROC, PR curves, confusion matrix
6. Optimise decision threshold via a business cost function
7. Top-decile lift analysis
8. SHAP global feature importance
9. Persist model artifacts

---
**Cost model:**  
- False Negative (missed churner): **\$200** per customer  
- False Positive (unnecessary outreach): **\$20** per customer  

Churn prevalence ≈ 26.5%, so we weight positive examples with `scale_pos_weight = 2.77`.

## 1. Imports

In [ ]:
from __future__ import annotations

import json
import os
import sys
import warnings
from datetime import datetime, timezone

os.environ.setdefault("LOKY_MAX_CPU_COUNT", "1")
os.environ.setdefault("MPLCONFIGDIR", "/tmp/mplconfig")
os.makedirs(os.environ["MPLCONFIGDIR"], exist_ok=True)
warnings.filterwarnings("ignore")

import joblib
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shap
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    average_precision_score,
    brier_score_loss,
    confusion_matrix,
    f1_score,
    precision_recall_curve,
    precision_score,
    recall_score,
    roc_auc_score,
    roc_curve,
    ConfusionMatrixDisplay,
)
from sklearn.model_selection import StratifiedKFold, train_test_split

# Add project root to path so src imports work
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
if BASE_DIR not in sys.path:
    sys.path.insert(0, BASE_DIR)

from src.feature_engineering import FEATURE_COLUMNS

print(f"Python: {sys.version.split()[0]}")
print(f"LightGBM: {lgb.__version__}")
print(f"SHAP: {shap.__version__}")
print(f"Feature columns ({len(FEATURE_COLUMNS)}): OK")

## 2. Load Features and Inspect Class Balance

In [ ]:
# ---------------------------------------------------------------------------
# Paths
# ---------------------------------------------------------------------------
CUSTOMER_FEATURES_PARQUET = os.path.join(BASE_DIR, "data", "processed", "customer_features.parquet")
MODELS_DIR = os.path.join(BASE_DIR, "models")
DOCS_DIR = os.path.join(BASE_DIR, "docs")

os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(DOCS_DIR, exist_ok=True)

TARGET_COLUMN = "Churn"
CUSTOMER_ID_COLUMN = "customerID"

# ---------------------------------------------------------------------------
# Load
# ---------------------------------------------------------------------------
df = pd.read_parquet(CUSTOMER_FEATURES_PARQUET, engine="pyarrow")
print(f"Loaded: {df.shape[0]:,} rows x {df.shape[1]} columns")

X = df[FEATURE_COLUMNS].copy()
y = df[TARGET_COLUMN].astype(int).copy()
customer_ids = df[CUSTOMER_ID_COLUMN].copy()

print(f"\nX shape: {X.shape}")
print(f"y shape: {y.shape}")
print()

# Class balance
class_counts = y.value_counts().sort_index()
print("Class distribution:")
print(f"  No Churn (0): {class_counts[0]:,}  ({class_counts[0]/len(y):.1%})")
print(f"  Churn    (1): {class_counts[1]:,}  ({class_counts[1]/len(y):.1%})")

fig, ax = plt.subplots(figsize=(5, 3.5))
bars = ax.bar(["No Churn", "Churn"], class_counts.values,
               color=["#4C72B0", "#DD8452"], edgecolor="white", linewidth=0.8)
for bar, count in zip(bars, class_counts.values):
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 20,
            f"{count:,}\n({count/len(y):.1%})", ha="center", va="bottom", fontsize=9)
ax.set_title("Class Balance — Churn Target", fontsize=12)
ax.set_ylabel("Customers")
ax.set_ylim(0, class_counts.max() * 1.18)
plt.tight_layout()
plt.show()

## 3. Train/Test Split with Class Distribution Check

In [ ]:
RANDOM_STATE = 42
TEST_SIZE = 0.20

X_train, X_test, y_train, y_test, ids_train, ids_test = train_test_split(
    X, y, customer_ids,
    test_size=TEST_SIZE,
    stratify=y,
    random_state=RANDOM_STATE,
)

# Reset indices for clean iloc access later
X_train = X_train.reset_index(drop=True)
X_test  = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test  = y_test.reset_index(drop=True)
ids_train = ids_train.reset_index(drop=True)
ids_test  = ids_test.reset_index(drop=True)

print(f"Train: {len(X_train):,} rows  |  Test: {len(X_test):,} rows")
print(f"Train churn rate: {y_train.mean():.1%}   Test churn rate: {y_test.mean():.1%}")

# Side-by-side bar chart
fig, axes = plt.subplots(1, 2, figsize=(9, 3.5))
for ax, y_split, title in zip(axes, [y_train, y_test], ["Train set", "Test set"]):
    counts = y_split.value_counts().sort_index()
    ax.bar(["No Churn", "Churn"], counts.values,
           color=["#4C72B0", "#DD8452"], edgecolor="white", linewidth=0.8)
    for i, (v, pct) in enumerate(zip(counts.values, counts.values / len(y_split))):
        ax.text(i, v + 10, f"{v:,}\n({pct:.1%})", ha="center", va="bottom", fontsize=9)
    ax.set_title(title)
    ax.set_ylabel("Customers")
    ax.set_ylim(0, counts.max() * 1.18)
plt.suptitle("Class Distribution by Split", fontsize=12, y=1.02)
plt.tight_layout()
plt.show()

## 4. Baseline Models and Cross-Validated AUC

In [ ]:
N_FOLDS = 5
SCALE_POS_WEIGHT = 2.77  # 73.5 / 26.5

skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)

naive_prob = float(y_train.mean())

naive_cv, lr_cv, lgbm_cv = [], [], []

for fold_i, (tr_idx, vl_idx) in enumerate(skf.split(X_train, y_train), 1):
    X_tr, X_vl = X_train.iloc[tr_idx], X_train.iloc[vl_idx]
    y_tr, y_vl = y_train.iloc[tr_idx], y_train.iloc[vl_idx]

    # Naive
    naive_cv.append(roc_auc_score(y_vl, np.full(len(y_vl), naive_prob)))

    # Logistic Regression
    lr_fold = LogisticRegression(C=1.0, max_iter=1000, random_state=RANDOM_STATE)
    lr_fold.fit(X_tr, y_tr)
    lr_cv.append(roc_auc_score(y_vl, lr_fold.predict_proba(X_vl)[:, 1]))

    # LightGBM
    lgbm_fold = lgb.LGBMClassifier(
        objective="binary", metric="auc",
        learning_rate=0.05, num_leaves=31,
        min_child_samples=20, feature_fraction=0.8,
        bagging_fraction=0.8, bagging_freq=5,
        scale_pos_weight=SCALE_POS_WEIGHT,
        n_estimators=300, random_state=RANDOM_STATE, verbose=-1,
    )
    lgbm_fold.fit(X_tr, y_tr)
    lgbm_cv.append(roc_auc_score(y_vl, lgbm_fold.predict_proba(X_vl)[:, 1]))

    print(f"  Fold {fold_i}: Naive={naive_cv[-1]:.4f}  LR={lr_cv[-1]:.4f}  LGBM={lgbm_cv[-1]:.4f}")

print()
cv_summary = pd.DataFrame({
    "Model": ["Naive Baseline", "Logistic Regression", "LightGBM"],
    "Mean CV AUC": [np.mean(naive_cv), np.mean(lr_cv), np.mean(lgbm_cv)],
    "Std CV AUC":  [np.std(naive_cv),  np.std(lr_cv),  np.std(lgbm_cv)],
})
cv_summary["Mean CV AUC"] = cv_summary["Mean CV AUC"].map("{:.4f}".format)
cv_summary["Std CV AUC"]  = cv_summary["Std CV AUC"].map("{:.4f}".format)
print(cv_summary.to_string(index=False))

## 5. LightGBM Final Training

In [ ]:
# ============================================================
# Hyperparameter configuration — edit here to experiment
# ============================================================
LGBM_PARAMS = dict(
    objective="binary",
    metric="auc",
    learning_rate=0.05,
    num_leaves=31,
    min_child_samples=20,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=5,
    scale_pos_weight=SCALE_POS_WEIGHT,
    n_estimators=300,
    random_state=RANDOM_STATE,
    verbose=-1,
)
# ============================================================

print("Training LightGBM on full training set...")
print(f"  Params: {LGBM_PARAMS}")

lgbm_model = lgb.LGBMClassifier(**LGBM_PARAMS)
lgbm_model.fit(X_train, y_train)

# Also train logistic regression final model
lr_model = LogisticRegression(C=1.0, max_iter=1000, random_state=RANDOM_STATE)
lr_model.fit(X_train, y_train)

print(f"  LightGBM trained: {lgbm_model.n_estimators_} trees")
print(f"  Logistic Regression trained: {X_train.shape[1]} features")

## 6. Probability Calibration with Isotonic Regression

In [ ]:
# Internal 80/20 split of training set for calibration
X_fit, X_cal, y_fit, y_cal = train_test_split(
    X_train, y_train,
    test_size=0.20,
    stratify=y_train,
    random_state=RANDOM_STATE,
)

# Refit LightGBM on 80% portion
lgbm_cal_inner = lgb.LGBMClassifier(**LGBM_PARAMS)
lgbm_cal_inner.fit(X_fit, y_fit)

# Score calibration hold-out
cal_raw = lgbm_cal_inner.predict_proba(X_cal)[:, 1]

# Fit isotonic regression
calibrator = IsotonicRegression(y_min=0.0, y_max=1.0, out_of_bounds="clip")
calibrator.fit(cal_raw, y_cal)

print(f"Calibration: fit={len(X_fit):,} rows, cal={len(X_cal):,} rows")
print(f"Calibrator type: {type(calibrator).__name__}")

# ---------------------------------------------------------------------------
# Plot: uncalibrated vs calibrated probability curves
# ---------------------------------------------------------------------------
from sklearn.calibration import calibration_curve

# Get test set scores from the FULL model (trained on all X_train)
test_raw_scores = lgbm_model.predict_proba(X_test)[:, 1]
test_cal_scores = calibrator.predict(test_raw_scores)

frac_pos_raw, mean_pred_raw = calibration_curve(y_test, test_raw_scores, n_bins=10)
frac_pos_cal, mean_pred_cal = calibration_curve(y_test, test_cal_scores, n_bins=10)

fig, ax = plt.subplots(figsize=(6, 5))
ax.plot([0, 1], [0, 1], "k--", linewidth=1, label="Perfect calibration")
ax.plot(mean_pred_raw, frac_pos_raw, "o-", color="#DD8452", label="LightGBM (uncalibrated)", markersize=5)
ax.plot(mean_pred_cal, frac_pos_cal, "s-", color="#4C72B0", label="LightGBM (isotonic)", markersize=5)
ax.set_xlabel("Mean Predicted Probability")
ax.set_ylabel("Fraction of Positives")
ax.set_title("Calibration Curve — Uncalibrated vs Isotonic")
ax.legend()
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
plt.tight_layout()
plt.show()

## 7. Evaluation: ROC Curve, PR Curve, Confusion Matrix

In [ ]:
# Score all models on test set
naive_scores  = np.full(len(y_test), float(y_train.mean()))
lr_scores     = lr_model.predict_proba(X_test)[:, 1]
lgbm_raw      = lgbm_model.predict_proba(X_test)[:, 1]
lgbm_scores   = calibrator.predict(lgbm_raw)

auc_naive = roc_auc_score(y_test, naive_scores)
auc_lr    = roc_auc_score(y_test, lr_scores)
auc_lgbm  = roc_auc_score(y_test, lgbm_scores)

pr_auc_lgbm = average_precision_score(y_test, lgbm_scores)
brier_lgbm  = brier_score_loss(y_test, lgbm_scores)

print(f"AUC-ROC  — Naive: {auc_naive:.4f} | LR: {auc_lr:.4f} | LightGBM (cal): {auc_lgbm:.4f}")
print(f"PR-AUC   — LightGBM: {pr_auc_lgbm:.4f}")
print(f"Brier    — LightGBM: {brier_lgbm:.6f}")

# ---------------------------------------------------------------------------
# ROC Curve
# ---------------------------------------------------------------------------
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

ax = axes[0]
for scores, label, color in [
    (naive_scores, f"Naive (AUC={auc_naive:.3f})",  "#929591"),
    (lr_scores,    f"Logistic (AUC={auc_lr:.3f})",   "#DD8452"),
    (lgbm_scores,  f"LightGBM (AUC={auc_lgbm:.3f})", "#4C72B0"),
]:
    fpr, tpr, _ = roc_curve(y_test, scores)
    ax.plot(fpr, tpr, label=label, linewidth=2, color=color)
ax.plot([0, 1], [0, 1], "k--", linewidth=1)
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curves")
ax.legend(fontsize=9)

# ---------------------------------------------------------------------------
# PR Curve
# ---------------------------------------------------------------------------
ax = axes[1]
prec, rec, _ = precision_recall_curve(y_test, lgbm_scores)
ax.plot(rec, prec, color="#4C72B0", linewidth=2, label=f"LightGBM (PR-AUC={pr_auc_lgbm:.3f})")
ax.axhline(y_test.mean(), color="#929591", linestyle="--", linewidth=1, label=f"Baseline ({y_test.mean():.3f})")
ax.set_xlabel("Recall")
ax.set_ylabel("Precision")
ax.set_title("Precision-Recall Curve — LightGBM (calibrated)")
ax.legend(fontsize=9)

plt.tight_layout()
plt.show()

# ---------------------------------------------------------------------------
# Confusion Matrix at 0.5 threshold
# ---------------------------------------------------------------------------
y_pred_05 = (lgbm_scores >= 0.5).astype(int)
print(f"\nMetrics @ threshold 0.5:")
print(f"  F1:        {f1_score(y_test, y_pred_05):.4f}")
print(f"  Precision: {precision_score(y_test, y_pred_05):.4f}")
print(f"  Recall:    {recall_score(y_test, y_pred_05):.4f}")

fig, ax = plt.subplots(figsize=(5, 4))
cm = confusion_matrix(y_test, y_pred_05)
disp = ConfusionMatrixDisplay(cm, display_labels=["No Churn", "Churn"])
disp.plot(ax=ax, colorbar=False, cmap="Blues")
ax.set_title("Confusion Matrix @ threshold 0.5")
plt.tight_layout()
plt.show()

## 8. Threshold Optimisation: Cost Curve

In [ ]:
COST_FN = 200   # missed churner
COST_FP = 20    # false alarm

thresholds = np.linspace(0.1, 0.9, 81)
costs = []
f1s = []

for thr in thresholds:
    y_pred = (lgbm_scores >= thr).astype(int)
    cm_thr = confusion_matrix(y_test, y_pred)
    fn = cm_thr[1, 0] if cm_thr.shape == (2, 2) else 0
    fp = cm_thr[0, 1] if cm_thr.shape == (2, 2) else 0
    costs.append(fn * COST_FN + fp * COST_FP)
    f1s.append(f1_score(y_test, y_pred, zero_division=0))

costs = np.array(costs)
f1s   = np.array(f1s)

best_idx = int(np.argmin(costs))
cost_optimal_threshold = float(thresholds[best_idx])
cost_optimal_value     = float(costs[best_idx])

print(f"Cost-optimal threshold: {cost_optimal_threshold:.2f}")
print(f"Total cost at optimal:  ${cost_optimal_value:,.0f}")
print(f"F1 at optimal:          {f1s[best_idx]:.4f}")

# Metrics at optimal threshold
y_pred_opt = (lgbm_scores >= cost_optimal_threshold).astype(int)
print(f"Precision:              {precision_score(y_test, y_pred_opt):.4f}")
print(f"Recall:                 {recall_score(y_test, y_pred_opt):.4f}")

# ---------------------------------------------------------------------------
# Cost curve plot
# ---------------------------------------------------------------------------
fig, ax1 = plt.subplots(figsize=(8, 4.5))
ax2 = ax1.twinx()

ax1.plot(thresholds, costs, color="#DD8452", linewidth=2, label="Total cost ($)")
ax1.axvline(cost_optimal_threshold, color="red", linestyle="--", linewidth=1.5,
            label=f"Optimal threshold = {cost_optimal_threshold:.2f}")
ax1.scatter([cost_optimal_threshold], [cost_optimal_value], color="red", s=80, zorder=5)
ax1.set_xlabel("Decision Threshold")
ax1.set_ylabel("Total Cost ($)", color="#DD8452")
ax1.tick_params(axis="y", labelcolor="#DD8452")

ax2.plot(thresholds, f1s, color="#4C72B0", linewidth=1.5, linestyle=":", label="F1 score")
ax2.set_ylabel("F1 Score", color="#4C72B0")
ax2.tick_params(axis="y", labelcolor="#4C72B0")

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, fontsize=9, loc="upper right")

ax1.set_title(f"Business Cost vs. Decision Threshold\n"
              f"(FN=${COST_FN}/customer, FP=${COST_FP}/customer)")
plt.tight_layout()
plt.show()

## 9. Top-Decile Lift Chart

In [ ]:
# Assign score deciles (1 = top-scored, 10 = lowest-scored)
decile_labels = pd.qcut(
    lgbm_scores,
    q=10,
    labels=[f"D{i}" for i in range(1, 11)],
    duplicates="drop",
)
decile_df = pd.DataFrame({
    "decile": decile_labels,
    "churn": y_test.values,
    "score": lgbm_scores,
})

# Sort deciles so D1 = highest-scored (most likely to churn)
decile_order = [f"D{i}" for i in range(1, 11)]
decile_stats = (
    decile_df.groupby("decile", observed=True)
    .agg(churn_rate=("churn", "mean"), n=("churn", "count"))
    .reindex(decile_order)
    .iloc[::-1]  # flip so highest-score (D10 from qcut) is on the left
    .reset_index()
)
decile_stats.index = [f"D{i}" for i in range(1, len(decile_stats) + 1)]

overall_churn_rate = float(y_test.mean())
lift = decile_stats["churn_rate"] / overall_churn_rate

print("Top-decile lift table:")
print(pd.DataFrame({"Decile": decile_stats.index,
                    "Churn Rate": decile_stats["churn_rate"].map("{:.1%}".format),
                    "Lift vs. Mean": lift.map("{:.2f}x".format)}).to_string(index=False))

# Capture rate: top-20% = deciles 1-2
top2_decile_rows = decile_df[decile_df["decile"].isin(decile_order[-2:])]
top20_capture = top2_decile_rows["churn"].sum() / y_test.sum()
print(f"\nTop-20% capture rate: {top20_capture:.1%} of all churners")

fig, ax = plt.subplots(figsize=(9, 4.5))
bars = ax.bar(
    decile_stats.index,
    decile_stats["churn_rate"],
    color=["#DD8452" if r >= overall_churn_rate * 1.5 else "#4C72B0" for r in decile_stats["churn_rate"]],
    edgecolor="white", linewidth=0.8,
)
ax.axhline(overall_churn_rate, color="black", linestyle="--", linewidth=1.5, label=f"Overall rate ({overall_churn_rate:.1%})")
for bar, rate in zip(bars, decile_stats["churn_rate"]):
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.003,
            f"{rate:.0%}", ha="center", va="bottom", fontsize=8)
ax.set_xlabel("Predicted Score Decile (D1 = highest-scored)")
ax.set_ylabel("Actual Churn Rate")
ax.set_title("Top-Decile Lift Chart — Actual Churn Rate by Score Decile")
ax.legend(fontsize=9)
plt.tight_layout()
plt.show()

## 10. SHAP Feature Importance

In [ ]:
print("Computing SHAP values on test set (using 200-row background)...")
bg_sample = X_train.sample(min(200, len(X_train)), random_state=RANDOM_STATE)
explainer = shap.TreeExplainer(lgbm_model, data=bg_sample)

# SHAP values on test set (for display)
shap_values = explainer.shap_values(X_test)

print(f"SHAP values shape: {np.array(shap_values).shape}")

# ---------------------------------------------------------------------------
# Global beeswarm plot
# ---------------------------------------------------------------------------
print("\nBeeswarm plot (global feature impact):")
shap.summary_plot(
    shap_values,
    X_test,
    plot_type="dot",
    max_display=20,
    show=True,
)
plt.title("SHAP Beeswarm — Global Feature Importance")
plt.tight_layout()
plt.show()

In [ ]:
# ---------------------------------------------------------------------------
# Top-10 feature importance bar chart (mean |SHAP|)
# ---------------------------------------------------------------------------
mean_abs_shap = np.abs(shap_values).mean(axis=0)
feature_importance = pd.Series(mean_abs_shap, index=FEATURE_COLUMNS).sort_values(ascending=False)

top10 = feature_importance.head(10)

fig, ax = plt.subplots(figsize=(8, 4.5))
ax.barh(top10.index[::-1], top10.values[::-1], color="#4C72B0", edgecolor="white", linewidth=0.8)
ax.set_xlabel("Mean |SHAP value|")
ax.set_title("Top-10 Features by Mean Absolute SHAP Value")
for i, (val, name) in enumerate(zip(top10.values[::-1], top10.index[::-1])):
    ax.text(val + 0.001, i, f"{val:.4f}", va="center", fontsize=8)
plt.tight_layout()
plt.show()

print("Top-10 features by mean |SHAP|:")
print(top10.map("{:.4f}".format).to_string())

## 11. Save Artifacts

In [ ]:
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(DOCS_DIR, exist_ok=True)

MODEL_ARTIFACT     = os.path.join(MODELS_DIR, "lgbm_churn.pkl")
CALIBRATOR_ARTIFACT = os.path.join(MODELS_DIR, "probability_calibrator.pkl")
SHAP_ARTIFACT      = os.path.join(MODELS_DIR, "shap_explainer.pkl")
METADATA_PATH      = os.path.join(MODELS_DIR, "metadata.json")

# Save LightGBM model
joblib.dump(lgbm_model, MODEL_ARTIFACT)
print(f"Saved: {MODEL_ARTIFACT}")

# Save calibrator
joblib.dump(calibrator, CALIBRATOR_ARTIFACT)
print(f"Saved: {CALIBRATOR_ARTIFACT}")

# Save SHAP explainer
joblib.dump(explainer, SHAP_ARTIFACT)
print(f"Saved: {SHAP_ARTIFACT}")

# Save metadata.json
metadata = {
    "feature_columns": FEATURE_COLUMNS,
    "train_rows": int(len(X_train)),
    "test_rows": int(len(X_test)),
    "metrics": {
        "auc_naive":   float(auc_naive),
        "auc_lr":      float(auc_lr),
        "auc_lgbm":    float(auc_lgbm),
        "pr_auc_lgbm": float(pr_auc_lgbm),
        "brier_lgbm":  float(brier_lgbm),
        "f1_05":       float(f1_score(y_test, (lgbm_scores >= 0.5).astype(int))),
        "precision_05": float(precision_score(y_test, (lgbm_scores >= 0.5).astype(int))),
        "recall_05":   float(recall_score(y_test, (lgbm_scores >= 0.5).astype(int))),
        "top20_lift":  float(top20_capture),
    },
    "cost_optimal_threshold": float(cost_optimal_threshold),
    "timestamp": datetime.now(timezone.utc).isoformat(),
}
with open(METADATA_PATH, "w", encoding="utf-8") as fh:
    json.dump(metadata, fh, indent=2)
print(f"Saved: {METADATA_PATH}")

print("\n--- Metadata summary ---")
print(json.dumps({k: v for k, v in metadata.items() if k != "feature_columns"}, indent=2))